In [1]:

! pip install python-dotenv
! pip install -qU langchain-openai
! pip install -qU langchain_community beautifulsoup4
! pip install faiss-cpu
! pip install -qU langchain-groq
! pip install langchain_core
! pip install neo4j

In [2]:

env_content = """
LANGSMITH_TRACING=true
LANGSMITH_ENDPOINT=https://api.smith.langchain.com
LANGSMITH_API_KEY=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
LANGSMITH_PROJECT=pr-minty-canvas-57
GROQ_API_KEY=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
NEO4J_URI=neo4j+s://9b6f374b.databases.neo4j.io
NEO4J_USERNAME=neo4j
NEO4J_PASSWORD=7G9uKu6OcVmXGfDdHFneBsWybNirn5tEuMPyM5u7Ofg
AURA_INSTANCEID=9b6f374b
AURA_INSTANCENAME=Instance01

"""

with open(".env", "w") as f:
    f.write(env_content.strip())

print(".env file created successfully!")

.env file created successfully!


In [3]:
import os
from dotenv import load_dotenv
load_dotenv()

True

In [4]:
from langchain_community.graphs import Neo4jGraph

graph = Neo4jGraph(
    url="neo4j+s://9b6f374b.databases.neo4j.io",  # Your Neo4j URI
    username="neo4j",                             # Your Neo4j username
    password="7G9uKu6OcVmXGfDdHFneBsWybNirn5tEuMPyM5u7Ofg"  # Your Neo4j password
)

graph

<ipython-input-4-31a367e3d2a5>:3: LangChainDeprecationWarning: The class `Neo4jGraph` was deprecated in LangChain 0.3.8 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-neo4j package and should be used instead. To use it run `pip install -U :class:`~langchain-neo4j` and import as `from :class:`~langchain_neo4j import Neo4jGraph``.
  graph = Neo4jGraph(


In [5]:
movie_query="""
LOAD CSV WITH HEADERS FROM
"https://raw.githubusercontent.com/tomasonjo/blog-datasets/refs/heads/main/movies/movies_small.csv" as row
MERGE (m:Movie {id: row.movieId})
SET m.released = date(row.released),
    m.title = row.title,
    m.imdbRating = toFloat(row.imdbRating)

FOREACH (director in split(row.director, "|") |
  MERGE (p:Person {name: trim(director)})
  MERGE (p)-[:DIRECTED]->(m))

FOREACH (actor in split(row.actors, "|") |
  MERGE (p:Person {name: trim(actor)})
  MERGE (p)-[:ACTED_IN]->(m))

FOREACH (genre in split(row.genres, "|") |
  MERGE (g:Genre {name: trim(genre)})
  MERGE (m)-[:IN_GENRE]->(g))

"""


In [6]:
graph.refresh_schema()
print(graph.schema)

Node properties:
Movie {id: STRING, released: DATE, title: STRING, imdbRating: FLOAT}
Person {name: STRING}
Genre {name: STRING}
Relationship properties:

The relationships:
(:Movie)-[:IN_GENRE]->(:Genre)
(:Person)-[:DIRECTED]->(:Movie)
(:Person)-[:ACTED_IN]->(:Movie)


In [7]:
from langchain_groq import ChatGroq
llm=ChatGroq(model_name="Gemma2-9b-It")
from langchain.chains import GraphCypherQAChain
chain=GraphCypherQAChain.from_llm(graph=graph,llm=llm,verbose=True,allow_dangerous_requests=True)
chain

GraphCypherQAChain(verbose=True, graph=<langchain_community.graphs.neo4j_graph.Neo4jGraph object at 0x7987e9f010d0>, cypher_generation_chain=LLMChain(verbose=False, prompt=PromptTemplate(input_variables=['question', 'schema'], input_types={}, partial_variables={}, template='Task:Generate Cypher statement to query a graph database.\nInstructions:\nUse only the provided relationship types and properties in the schema.\nDo not use any other relationship types or properties that are not provided.\nSchema:\n{schema}\nNote: Do not include any explanations or apologies in your responses.\nDo not respond to any questions that might ask anything else than for you to construct a Cypher statement.\nDo not include any text except the generated Cypher statement.\n\nThe question is:\n{question}'), llm=ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x7987dd19b6d0>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x7987dcd9f190>, model_name='Gemma2-9b-I

In [8]:
response=chain.invoke({"query":"Who was the director of the movie Casino"})
response



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (m:Movie{title:"Casino"})-[r:DIRECTED]->(p:Person) RETURN p.name
Full Context:
[]

> Finished chain.


{'query': 'Who was the director of the movie Casino',
 'result': "I don't know the answer. \n"}

In [9]:
response=chain.invoke({"query":"Who were the actors of the movie Casino"})
response



> Entering new GraphCypherQAChain chain...
Generated Cypher:
cypher
MATCH (m:Movie {title:"Casino"})-[:ACTED_IN]->(p:Person)
RETURN p.name

Full Context:
[]

> Finished chain.


{'query': 'Who were the actors of the movie Casino',
 'result': "I don't know the answer. \n"}

In [10]:
response=chain.invoke({"query":"How many artists are there?"})
response



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (p:Person) RETURN count(p) 

Full Context:
[{'count(p)': 1239}]

> Finished chain.


{'query': 'How many artists are there?',
 'result': 'There are 1239 artists. \n'}

In [11]:
response=chain.invoke({"query":"How many movies has Tom Hanks acted in"})
response



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (p:Person {name: "Tom Hanks"})<-[:ACTED_IN]-(m:Movie)
RETURN count(m)

Full Context:
[{'count(m)': 0}]

> Finished chain.


{'query': 'How many movies has Tom Hanks acted in',
 'result': "I don't know the answer. \n"}